# TensorFlow ArcFace

### Ref https://github.com/peteryuX/arcface-tf2

In [1]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from models import ArcFaceModel
# from losses import SoftmaxLoss
from losses import softmax_loss
import dataset
import tensorflow as tf
import os
import logging

tf.get_logger().setLevel(logging.ERROR)
# os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [2]:
!echo $CUDA_VISIBLE_DEVICES
print("go!")

2
go!


In [3]:
### IJB-C Dataset

# batch_size = 128
# input_size = 112
# embd_shape = 512
# head_type = 'ArcHead'
# backbone_type = 'MobileNetV2'
# w_decay=5e-4
# num_classes = 3584 
# base_lr = 0.01
# dataset_len = 13033 
# epochs = 100
# steps_per_epoch = dataset_len // batch_size

### MS1M dataset

batch_size = 128 # Initially 128
input_size = 112
embd_shape = 512
head_type = 'CosHead'
# head_type = 'ArcHead'
backbone_type = 'ResNet50' # MobileNetV2, ResNet50, EfficientNetLite0~6
# backbone_type = 'MobileNetV2' # MobileNetV2, ResNet50, EfficientNetLite0~6
w_decay=5e-4
num_classes = 85742 
dataset_len = 5822653 
base_lr = 0.01 # initially 0.01
epochs = 20
save_steps = 1000
train_size = int(0.8 * dataset_len)
print(train_size)
steps_per_epoch = train_size // batch_size
print(steps_per_epoch)
val_size = dataset_len - train_size
print(val_size)
validation_steps = val_size // batch_size
print(validation_steps)
steps = 1
is_ccrop=False
binary_img=True
is_Adam = False

dgx = True

4658122
36391
1164531
9097


In [4]:
# train_data_dir = "/raid/workspace/jbpark/IJB-C_Asian/"
# tfrecord_name = train_data_dir+'ijbc_bin.tfrecord'
if dgx:
    train_data_dir = "/raid/workspace/jbpark/ms1m/"
    tfrecord_name = train_data_dir+'ms1m_bin.tfrecord'
else:
    train_data_dir = "/hd/jbpark/dataset/ms1m/"
    tfrecord_name = train_data_dir+'ms1m_bin.tfrecord'


train_dataset, val_dataset = dataset.load_tfrecord_dataset(
    tfrecord_name, batch_size, train_size=train_size, binary_img=binary_img, 
    is_ccrop=is_ccrop)

if dgx:
    strategy = tf.distribute.MirroredStrategy(devices=["/gpu:2"])
else:
    strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0"])
#     strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])

with strategy.scope():
    model = ArcFaceModel(size=input_size,
                             backbone_type=backbone_type,
                             num_classes=num_classes,
                             head_type=head_type,
                             embd_shape=embd_shape,
                             w_decay=w_decay,
                             training=True)
    model.summary()

    learning_rate = tf.constant(base_lr)
    if is_Adam:
        optimizer = tf.keras.optimizers.Adam(
            learning_rate=learning_rate, momentum=0.9, nesterov=True)
    else:
        optimizer = tf.keras.optimizers.SGD(
            learning_rate=learning_rate)

    loss_fn = softmax_loss

    model.compile(optimizer=optimizer, loss=loss_fn)

Model: "arcface_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        [(None, 112, 112, 3) 0                                            
__________________________________________________________________________________________________
resnet50 (Functional)           (None, 4, 4, 2048)   23587712    input_image[0][0]                
__________________________________________________________________________________________________
OutputLayer (Functional)        (None, 512)          16787968    resnet50[0][0]                   
__________________________________________________________________________________________________
label (InputLayer)              [(None,)]            0                                            
______________________________________________________________________________________

In [ ]:
from pathlib import Path

if dgx:
    base_dir = "/raid/workspace/honghee/FaceRecognition/"
    if backbone_type == 'ResNet50':
        save_name = "ms1m_resnet50_check/"
    elif backbone_type == 'MobileNetV2':
        save_name = "ms1m_mobilenet_check/"
else:
    base_dir = "/hd/honghee/models/"
    save_name = "ms1m_check/"
    
if head_type == 'CosHead':
    head = "Cosface/"
elif head_type == 'ArcHead':
    head = "Arcface/"
else:
    head = "Sphereface/"

if is_Adam:
    version = "Adam"
else:
    version = "SGD"

Path(base_dir + 'checkpoints/w_tfidentity/' + save_name + head + version).mkdir(parents=True, exist_ok=True)
# Path(base_dir + 'checkpoints/w_tfidentity/' + save_name + version).mkdir(parents=True, exist_ok=True)

### MS1M dataset
tb_callback = TensorBoard(log_dir='logs/arcface/',
                                  update_freq = batch_size * 5,
                                  profile_batch=0)
tb_callback._total_batches_seen = steps
tb_callback._samples_seen = steps * batch_size
mc_callback = ModelCheckpoint(
            base_dir + 'checkpoints/w_tfidentity/' + save_name + head + version +'/e_{epoch}_l_{loss}.ckpt',
            save_freq = save_steps, verbose=1,
            save_weights_only=True)
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=0.1, min_lr=0.001)
callbacks = [mc_callback, tb_callback,reduce_lr]

### IJB-C Dataset
# callbacks = [
#     ModelCheckpoint(
#         base_dir+"checkpoints/"+save_name+".ckpt", 
# #         monitor='val_accuracy', 
#         monitor='loss', 
#         verbose=1, 
#         save_best_only=True, 
#         save_weights_only = True,
#         mode='min'
#     ),
#     EarlyStopping(
# #         monitor='val_accuracy', 
#         monitor='loss', 
#         patience=15, 
#         min_delta=0.001, 
#         mode='min'
#     )
# ]

model.fit(
    train_dataset, 
    validation_data=val_dataset,
    epochs=epochs, 
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=callbacks
)

Epoch 1/20
 1000/36391 [..............................] - ETA: 59:50 - loss: 47.6697
Epoch 00001: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/Cosface/SGD/e_1_l_47.4111328125.ckpt
 2000/36391 [>.............................] - ETA: 58:41 - loss: 47.3589
Epoch 00001: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/Cosface/SGD/e_1_l_46.67818069458008.ckpt
 3000/36391 [=>............................] - ETA: 57:04 - loss: 47.0020
Epoch 00001: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/Cosface/SGD/e_1_l_45.89635467529297.ckpt
 4000/36391 [==>...........................] - ETA: 55:11 - loss: 46.6291
Epoch 00001: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/Cosface/SGD/e_1_l_45.129459381103516.ckpt
 5000/36391 [===>..........................] - ETA: 53:06 - loss: 46.2551
Ep

### Resume training with latest checkpoint

In [4]:
from glob import glob
if dgx:
    base_dir = "/raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/"
    # save_name = "ms1m_mobilenet_check/SGD/*"
    save_name = "ms1m_resnet50_check/SGD/*"
else:
    base_dir = "/hd/honghee/models/checkpoints/w_tfidentity/"
    save_name = "ms1m_resnet50/SGD/*"
file_list = []
for files in glob(base_dir+save_name):
    file_list.append(files.split('/')[-1].split('l_')[-1])
file_list.sort()

load_file_name = []
for files in glob(base_dir+save_name):
    if file_list[0] == files.split('/')[-1].split('l_')[-1]:
        load_file_name = files
best_checkpoint = load_file_name.split('.data')[0]
initial_epoch = int(load_file_name.split('e_')[-1].split('_')[0])-1
print(initial_epoch)
print(best_checkpoint)

0
/raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/SGD/e_1_l_29.518735885620117.ckpt


In [5]:
from pathlib import Path

if dgx:
    base_dir = "/raid/workspace/honghee/FaceRecognition/"
    # save_name = "ms1m_mobilenet_check/"
    save_name = "ms1m_resnet50_check/"
else:
    base_dir = "/hd/honghee/models/"
    save_name = "ms1m_check/"

if is_Adam:
    version = "Adam"
else:
    version = "SGD"

Path(base_dir + 'checkpoints/w_tfidentity/' + save_name + version).mkdir(parents=True, exist_ok=True)

# train_data_dir = "/raid/workspace/jbpark/IJB-C_Asian/"
# tfrecord_name = train_data_dir+'ijbc_bin.tfrecord'

if dgx:
    strategy = tf.distribute.MirroredStrategy(devices=["/gpu:2"])
else:
    strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0"])
#     strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])

if dgx:
    train_data_dir = "/raid/workspace/jbpark/ms1m/"
    tfrecord_name = train_data_dir+'ms1m_bin.tfrecord'
else:
    train_data_dir = "/hd/jbpark/dataset/ms1m/"
    tfrecord_name = train_data_dir+'ms1m_bin.tfrecord'


train_dataset, val_dataset = dataset.load_tfrecord_dataset(
    tfrecord_name, batch_size, train_size=train_size, binary_img=binary_img, 
    is_ccrop=is_ccrop)

with strategy.scope():

    model = ArcFaceModel(size=input_size,
                             backbone_type=backbone_type,
                             num_classes=num_classes,
                             head_type=head_type,
                             embd_shape=embd_shape,
                             w_decay=w_decay,
                             training=True)
    model.load_weights(best_checkpoint)

    learning_rate = tf.constant(base_lr)
    if is_Adam:
        optimizer = tf.keras.optimizers.Adam(
            learning_rate=learning_rate, momentum=0.9, nesterov=True)
    else:
        optimizer = tf.keras.optimizers.SGD(
            learning_rate=learning_rate)

    loss_fn = softmax_loss

    model.compile(optimizer=optimizer, loss=loss_fn)
    model.summary()
    
tb_callback = TensorBoard(log_dir='logs/arcface/',
                                  update_freq = batch_size * 5,
                                  profile_batch=0)
tb_callback._total_batches_seen = steps
tb_callback._samples_seen = steps * batch_size
mc_callback = ModelCheckpoint(
            base_dir + 'checkpoints/w_tfidentity/' + save_name + version +'/e_{epoch}_l_{loss}.ckpt',
            save_freq = save_steps, verbose=1,
            save_weights_only=True)
# reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
#                               patience=0.1, min_lr=0.001)
# callbacks = [mc_callback, tb_callback,reduce_lr]
callbacks = [mc_callback, tb_callback]

Model: "arcface_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        [(None, 112, 112, 3) 0                                            
__________________________________________________________________________________________________
resnet50 (Functional)           (None, 4, 4, 2048)   23587712    input_image[0][0]                
__________________________________________________________________________________________________
OutputLayer (Functional)        (None, 512)          16787968    resnet50[0][0]                   
__________________________________________________________________________________________________
label (InputLayer)              [(None,)]            0                                            
______________________________________________________________________________________

In [6]:
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs, 
    initial_epoch=initial_epoch,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=callbacks
)

Epoch 1/20
 1000/36391 [..............................] - ETA: 1:02:33 - loss: 18.9776
Epoch 00001: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/SGD/e_1_l_18.945175170898438.ckpt
 2000/36391 [>.............................] - ETA: 1:01:50 - loss: 18.9296
Epoch 00001: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/SGD/e_1_l_18.796144485473633.ckpt
 3000/36391 [=>............................] - ETA: 1:00:46 - loss: 18.8625
Epoch 00001: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/SGD/e_1_l_18.663278579711914.ckpt
 4000/36391 [==>...........................] - ETA: 59:12 - loss: 18.7969
Epoch 00001: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/SGD/e_1_l_18.541412353515625.ckpt
 5000/36391 [===>..........................] - ETA: 57:25 - loss: 18.7337
Epoch 00001: saving m

 1609/36391 [>.............................] - ETA: 57:38 - loss: 14.1621
Epoch 00002: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/SGD/e_2_l_14.145465850830078.ckpt
 2609/36391 [=>............................] - ETA: 56:57 - loss: 14.1415
Epoch 00002: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/SGD/e_2_l_14.082281112670898.ckpt
 3609/36391 [=>............................] - ETA: 55:37 - loss: 14.1191
Epoch 00002: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/SGD/e_2_l_14.046380996704102.ckpt
 4609/36391 [==>...........................] - ETA: 54:08 - loss: 14.0980
Epoch 00002: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/SGD/e_2_l_13.999074935913086.ckpt
 5609/36391 [===>..........................] - ETA: 52:31 - loss: 14.0766
Epoch 00002: saving model to /raid/wor

 2218/36391 [>.............................] - ETA: 57:54 - loss: 12.2328
Epoch 00003: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/SGD/e_3_l_12.185358047485352.ckpt
 3218/36391 [=>............................] - ETA: 56:22 - loss: 12.2118
Epoch 00003: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/SGD/e_3_l_12.150927543640137.ckpt
 4218/36391 [==>...........................] - ETA: 54:22 - loss: 12.1950
Epoch 00003: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/SGD/e_3_l_12.12775993347168.ckpt
 5218/36391 [===>..........................] - ETA: 51:54 - loss: 12.1792
Epoch 00003: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/SGD/e_3_l_12.097689628601074.ckpt
 6218/36391 [====>.........................] - ETA: 49:44 - loss: 12.1644
Epoch 00003: saving model to /raid/work

 2827/36391 [=>............................] - ETA: 1:09:04 - loss: 10.9999
Epoch 00004: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/SGD/e_4_l_10.963507652282715.ckpt
 3827/36391 [==>...........................] - ETA: 1:07:04 - loss: 10.9889
Epoch 00004: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/SGD/e_4_l_10.951705932617188.ckpt
 4827/36391 [==>...........................] - ETA: 1:04:47 - loss: 10.9786
Epoch 00004: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/SGD/e_4_l_10.929407119750977.ckpt
 5827/36391 [===>..........................] - ETA: 1:02:33 - loss: 10.9685
Epoch 00004: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/SGD/e_4_l_10.90965461730957.ckpt
 6827/36391 [====>.........................] - ETA: 1:00:33 - loss: 10.9587
Epoch 00004: saving model to 

 3436/36391 [=>............................] - ETA: 1:06:12 - loss: 10.1376
Epoch 00005: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/SGD/e_5_l_10.098734855651855.ckpt
 4436/36391 [==>...........................] - ETA: 1:04:25 - loss: 10.1280
Epoch 00005: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/SGD/e_5_l_10.08542537689209.ckpt
 5436/36391 [===>..........................] - ETA: 1:02:36 - loss: 10.1184
Epoch 00005: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/SGD/e_5_l_10.068511962890625.ckpt
 6436/36391 [====>.........................] - ETA: 1:00:43 - loss: 10.1099
Epoch 00005: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/SGD/e_5_l_10.060005187988281.ckpt
 7436/36391 [=====>........................] - ETA: 58:40 - loss: 10.1017
Epoch 00005: saving model to /r

...

10135/36391 [=======>......................] - ETA: 40:42 - loss: 6.3124
Epoch 00016: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/SGD/e_16_l_6.275978088378906.ckpt
11135/36391 [========>.....................] - ETA: 39:08 - loss: 6.3090
Epoch 00016: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/SGD/e_16_l_6.274550437927246.ckpt
12135/36391 [=========>....................] - ETA: 37:36 - loss: 6.3060
Epoch 00016: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/SGD/e_16_l_6.272220134735107.ckpt
13135/36391 [=========>....................] - ETA: 36:03 - loss: 6.3034
Epoch 00016: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/SGD/e_16_l_6.269269943237305.ckpt
14135/36391 [==========>...................] - ETA: 34:29 - loss: 6.3008
Epoch 00016: saving model to /raid/workspac

10744/36391 [=======>......................] - ETA: 40:10 - loss: 6.1400
Epoch 00017: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/SGD/e_17_l_6.104877471923828.ckpt
11744/36391 [========>.....................] - ETA: 38:37 - loss: 6.1370
Epoch 00017: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/SGD/e_17_l_6.1038336753845215.ckpt
12744/36391 [=========>....................] - ETA: 37:03 - loss: 6.1344
Epoch 00017: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/SGD/e_17_l_6.1013312339782715.ckpt
13744/36391 [==========>...................] - ETA: 35:28 - loss: 6.1319
Epoch 00017: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/SGD/e_17_l_6.10023307800293.ckpt
14744/36391 [===========>..................] - ETA: 33:51 - loss: 6.1297
Epoch 00017: saving model to /raid/workspa

11353/36391 [========>.....................] - ETA: 39:03 - loss: 5.9806
Epoch 00018: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/SGD/e_18_l_5.9498748779296875.ckpt
12353/36391 [=========>....................] - ETA: 37:29 - loss: 5.9781
Epoch 00018: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/SGD/e_18_l_5.949118614196777.ckpt
13353/36391 [==========>...................] - ETA: 35:55 - loss: 5.9758
Epoch 00018: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/SGD/e_18_l_5.945194244384766.ckpt
14353/36391 [==========>...................] - ETA: 34:21 - loss: 5.9737
Epoch 00018: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/SGD/e_18_l_5.942856788635254.ckpt
15353/36391 [===========>..................] - ETA: 32:48 - loss: 5.9716
Epoch 00018: saving model to /raid/workspa

11962/36391 [========>.....................] - ETA: 49:28 - loss: 5.8391
Epoch 00019: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/SGD/e_19_l_5.807738304138184.ckpt
12962/36391 [=========>....................] - ETA: 47:25 - loss: 5.8364
Epoch 00019: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/SGD/e_19_l_5.803159713745117.ckpt
13962/36391 [==========>...................] - ETA: 45:12 - loss: 5.8340
Epoch 00019: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/SGD/e_19_l_5.801770210266113.ckpt
14962/36391 [===========>..................] - ETA: 42:58 - loss: 5.8317
Epoch 00019: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/SGD/e_19_l_5.798458576202393.ckpt
15962/36391 [============>.................] - ETA: 40:44 - loss: 5.8295
Epoch 00019: saving model to /raid/workspac

12571/36391 [=========>....................] - ETA: 41:15 - loss: 5.7053
Epoch 00020: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/SGD/e_20_l_5.672054767608643.ckpt
13571/36391 [==========>...................] - ETA: 39:32 - loss: 5.7028
Epoch 00020: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/SGD/e_20_l_5.6705641746521.ckpt
14571/36391 [===========>..................] - ETA: 37:50 - loss: 5.7005
Epoch 00020: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/SGD/e_20_l_5.668464660644531.ckpt
15571/36391 [===========>..................] - ETA: 36:08 - loss: 5.6983
Epoch 00020: saving model to /raid/workspace/honghee/FaceRecognition/checkpoints/w_tfidentity/ms1m_resnet50_check/SGD/e_20_l_5.664841175079346.ckpt
16571/36391 [============>.................] - ETA: 34:24 - loss: 5.6962
Epoch 00020: saving model to /raid/workspace/